In [1]:
# Prefer interactive ipympl; fall back to inline if unavailable
USING_WIDGET = False
try:
    %matplotlib widget
    USING_WIDGET = True
except Exception:
    %matplotlib inline

import os, h5py, numpy as np
import matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display


In [2]:
# ─────────────────────────────────────────────────────────────────────
# CONFIG — update this path if needed
# ─────────────────────────────────────────────────────────────────────
RESULTS_DIR = r"D:/Results/09052025_AE1M_Conv2DTranspose"
TSNE_H5     = os.path.join(RESULTS_DIR, "tsne_embeddings.h5")

# big canvases can be heavy; start moderate and bump if you like
FIGSIZE_IN  = (5, 5)   # try (50, 50) if your notebook handles it
DPI         = 150

# points
POINT_SIZE  = 0.4
POINT_ALPHA = 0.5

# Helpful for huge scatters
plt.rcParams['agg.path.chunksize'] = 10_000


In [3]:
if not os.path.isfile(TSNE_H5):
    raise FileNotFoundError(TSNE_H5)

with h5py.File(TSNE_H5, "r") as h5:
    xy = np.asarray(h5["/tsne_2d"][...], dtype=np.float32)

xy.shape


(1061277, 2)

In [4]:

def _nice_limits(xy: np.ndarray, pad_frac: float = 0.02):
    x_min, y_min = np.min(xy, axis=0)
    x_max, y_max = np.max(xy, axis=0)
    dx, dy = x_max - x_min, y_max - y_min
    px, py = dx * pad_frac, dy * pad_frac
    return (x_min - px, x_max + px), (y_min - py, y_max + py)

plt.ioff()  # ← prevent Jupyter from auto-showing the figure (fixes double-plot)

# Figure
fig, ax = plt.subplots(figsize=FIGSIZE_IN, dpi=DPI)
(xlim, ylim) = _nice_limits(xy)

pts = ax.scatter(
    xy[:, 0], xy[:, 1],
    s=POINT_SIZE, c="k", alpha=POINT_ALPHA, linewidths=0
)
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set_aspect("equal", adjustable="box")
ax.set_title("t-SNE — all points (interactive)")
ax.set_xticks([]); ax.set_yticks([])
ax.grid(False)

# ipympl niceties (if available)
if USING_WIDGET and hasattr(fig, "canvas"):
    try:
        fig.canvas.header_visible = False
        fig.canvas.toolbar_visible = True
        fig.canvas.resizable = True
    except Exception:
        pass

# Controls
w_size  = W.FloatLogSlider(value=POINT_SIZE, base=10, min=-2.0, max=1.0, step=0.01,
                           description="Point size", readout_format=".3f", layout=W.Layout(width="400px"))
w_alpha = W.FloatSlider(value=POINT_ALPHA, min=0.05, max=1.0, step=0.05,
                        description="Alpha", layout=W.Layout(width="400px"))
w_reset = W.Button(description="Reset view", icon="refresh")

def on_size_change(_):
    pts.set_sizes(np.full(xy.shape[0], w_size.value))
    if USING_WIDGET: fig.canvas.draw_idle()

def on_alpha_change(_):
    pts.set_alpha(w_alpha.value)
    if USING_WIDGET: fig.canvas.draw_idle()

def on_reset_clicked(_):
    (xlim, ylim) = _nice_limits(xy)
    ax.set_xlim(*xlim); ax.set_ylim(*ylim)
    if USING_WIDGET: fig.canvas.draw_idle()

w_size.observe(on_size_change, names="value")
w_alpha.observe(on_alpha_change, names="value")
w_reset.on_click(on_reset_clicked)

toolbar = W.HBox([w_size, w_alpha, w_reset])



In [5]:
#| label: fig:ae1m-t-SNE
#| caption: "."

# Display (single output)
if USING_WIDGET:
    from IPython.display import display
    display(W.VBox([toolbar, fig.canvas]))
else:
    from IPython.display import display, HTML
    display(W.VBox([HTML("<b>Note:</b> ipympl not active — plot is static. "
                         "Run <code>%matplotlib widget</code> if available."), toolbar]))
    plt.show()
